In [1]:
rootf = FOLDER_NAME ## Here insert your rbm-mhc folder path. e.g. '/home/user/rbm-mhc' ##


## imports ##
import sys, os, pickle
sys.path.append(rootf + '/PGM/source/')
sys.path.append(rootf + '/PGM/utilities/')
sys.path.append(rootf + '/Align_utils/')
from common_imports import set_num_threads
set_num_threads(1)
import numpy as np
import pandas as pd
import rbm,utilities
import Proteins_utils, RBM_utils, utilities,sequence_logo,plots_utils
import importlib
import butils
import sequence_logo
import matplotlib.pyplot as plt

/home/barbara/rbm-mhc/PGM/source/numba_utilities.py:1124: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float32, 2d, F), array(float32, 2d, A))
  dmean_v_dw = np.dot(s1.T, V)
/home/barbara/rbm-mhc/PGM/source/numba_utilities.py:1125: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float32, 2d, F), array(float32, 2d, A))
  dvar_e_dw = np.dot(s2.T, V)
/home/barbara/rbm-mhc/PGM/source/numba_utilities.py:1126: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float32, 2d, F), array(float32, 2d, A))
  tmp3 = np.dot(s3.T, V)
/home/barbara/rbm-mhc/PGM/source/numba_utilities.py:961: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float32, 1d, A), array(float32, 2d, A))
  mean_V = np.dot(weights, V) / sum_weights


KeyboardInterrupt: 

Here, we read the table where peptides are assigned a HLA-I type by RBM-MHC and we 
visualize motifs corresponding to peptides predicted by RBM-MHC to be of the same 
HLA binding specificity as in Fig. 1b. We also print peptides of each HLA-I type in different files.

In [ ]:
names = 'sample_file' ## here insert the sample file name ##
nameo = 'string_output' ## here insert the string used to name output files ##
out_fold = rootf + '/output_folder'

hlas = ['HLA-A*01:01', 'HLA-A*03:01', 'HLA-B*07:02', 'HLA-B*08:01', 'HLA-C*07:01', 'HLA-C*07:02']
nc = len(hlas)

filename_motif = out_fold + '/' + names + '_classification_' + nameo + '.txt'
table1 = pd.read_csv(filename_motif , sep='\t')
table2 = table1
## This line allows to exclude data with labels from IEDB ## 
# table2 = table1[table1['Training']==0]

CC = 21 # number of symbols: amino acids + gap
SA = 9 # length of peptides

pseudocount = 0.000001

fig, axess = plt.subplots(6,1)
fig.set_figheight(7)
fig.set_figwidth(4)
fig.subplots_adjust(wspace=0.5, hspace = 1.1, bottom = 0.2)
st=12
sl=st

seqs_all=[]
for h in range(nc):
    seqs_allh = list(table1[table1['RBM-MHC label'] == h]['Data Core'].values)
    Nrbm=len(seqs_allh)
    seqs_all.append(seqs_allh)
    sel_n = butils.convert_number(seqs_allh)
    pwm_rbm = utilities.average(sel_n, c = CC);
        
    for i in range(SA):
        for a in range(CC):
            if pwm_rbm[i,a] < pseudocount:
                pwm_rbm[i,a] = pseudocount  
            
    ax1= axess[h]
    titlep =  hlas[h] + ' motif (# seqs. = ' + str(Nrbm) + ')'
    sequence_logo.Sequence_logo(pwm_rbm, ax = ax1, figsize=(12,3.5), ylabel= 'bits', title = titlep, show=True, ticks_every=1, ticks_labels_size=sl, title_size=st)

plt.tight_layout()
namefig = out_fold + '/motifs_reconstructed.png'
fig.savefig(namefig,dpi=300)
plt.close()

for cl in range(nc):
    name_fi = out_fold + '/peptides_'+ nameo + '_' + str(hlas[cl])
    with open(name_fi  + '.txt', 'w') as out_f:
        for r in range(len(seqs_all[cl])):
            out_f.write(seqs_all[cl][r] + '\n')

Here, we upload the trained RBM model, we visualize its weights and 
examples of inputs to hidden units, similarly to Fig. 1b and Supplementary Figs. 3-4.

In [ ]:
name_r = out_fold + '/model_' + nameo + '.data'
RBM = RBM_utils.loadRBM(name_r)
hu = RBM.n_h ## hidden units of the model ##

## figure with weights ## 
fig, axess = plt.subplots(10,1)
fig.set_figheight(11)
fig.set_figwidth(4)
fig.subplots_adjust(wspace=0.5, hspace = 1., bottom = 0.2)
st=12
sl=st

for h in range(hu):           
    ax1= axess[h]
    titlep = ''
    sequence_logo.Sequence_logo(RBM.weights[h], ax = ax1, figsize=(12,3.5), ylabel = r'W_' + str(h + 1), title=' ', ticks_every = 1,ticks_labels_size=sl,title_size=st)    
plt.tight_layout()

namefig = out_fold + '/weights.png'
fig.savefig(namefig,  bbox_inches = 'tight', pad_inches = 0, dpi = 300)
plt.close()

colors = ['r','orange', 'deepskyblue', 'blue', 'springgreen', 'forestgreen']

s2 = 18
sl = 12
list_ix=[0,1] ## example of hidden units, but any could be selected ##
list_iy=[2,3] ## example of hidden units, but any could be selected ##
s1 = 1.0

for ix in list_ix:
    for iy in list_iy:
        fig, ax = plt.subplots()
        fig.set_figwidth(5)
        fig.set_figheight(5)

        for h in range(nc):
            sel_n = butils.convert_number(seqs_all[h])
            In = RBM.vlayer.compute_output(sel_n, RBM.weights)
            ax.scatter(In[:,ix],In[:,iy], c = colors[h], s = s1, label = hlas[h])

        ax.set_xlabel('Input to h_' + str(ix+1), fontsize=s2)
        ax.set_ylabel('Input to h_' + str(iy+1), fontsize=s2)
        ax.tick_params(axis='both', which='major', labelsize = s2)
        plt.tight_layout()
        plt.legend(fontsize=10,markerscale=2,frameon=False, loc ='upper right')
        pathfig = out_fold + '/Input_' + str(ix+1) + '_vs_Input_' + str(iy+1) + '.png'
        fig.savefig(pathfig,  bbox_inches = 'tight', pad_inches = 0, dpi = 300)
        plt.close() 

Here, we read the table of scored peptides and we rank them according to the 
RBM-MHC presentation score

In [ ]:
namep = 'peptides-to-score' ## here insert the name of the file with peptides to score ##
fn = out_fold + '/scoring/' + namep + '_scored_' + nameo + '.txt'
scores_out = pd.read_csv(fn, sep = '\t')

llt = scores_out['Total RBM-MHC score'].values
pred = scores_out['Pred. HLA class'].values
seqs = list(scores_out['Sequences'].values)
length = scores_out['Length'].values

Scores_list = zip(seqs, llt, pred, length)
Srank = sorted(sorted(Scores_list, key = lambda x : x[3]), key = lambda x : x[1], reverse = True)

data_ranked = {'Peptide':list(np.array(Srank)[:,0]),
        'RBM-MHC score': list(np.array(Srank)[:,1]),
        'Predicted HLA-I': list(np.array(Srank)[:,2]),
        'Peptide Length': list(np.array(Srank)[:,3])      
        }
pd.DataFrame (data_ranked, columns = ['Peptide','RBM-MHC score','Predicted HLA-I', 'Peptide Length'])